# Sequence to Sequence model to extract summary,objective from resume (input: entire resume)


## Trained data 40 resumes and 10 testing resumes to validate

### Input:

### Output:


In [1]:
from __future__ import print_function
import csv,os
import collections
from keras.models import Model,load_model
from keras.layers import Input,LSTM,Dense
from keras.callbacks import ModelCheckpoint
import numpy as np

batch_size=10
epochs=50
latent_dim=128
data_path="/home/santhosh/resumes_folder/custom_annotator/annotator-server/static/files/Data_Tracter_Resumes_in_TXT/csv_1"
import os

/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## vectorizing the data

In [2]:
input_resumes=[]
output_summary=[]
input_tokens=collections.Counter()
output_tokens=set()
files=os.listdir(data_path)
files.sort()
file_count=0
dummy_file_count=0
for file in files:
    with open(data_path+'/'+file,'r') as csv_file:
        reader=csv.reader(csv_file)
        input_text=" "
        output_text=" "
        count=0
        flag=0
        for Input_text,output in reader:
            Input_text=Input_text.strip().lower()
            input_text+=Input_text+' \n'
            if(output=='1'):
                flag=1
                output_text+=Input_text+' \n' 
            if count==15:
                break
            count+=1
        
        if  flag==0:
            dummy_file_count+=1
        
        if flag==0 and dummy_file_count>5:
            print(file)
            continue
        elif flag==0:
            print(file_count)

    # We use "<SOL>" as the "start sequence" character
    # for the targets, and "<EOL>" as "end sequence" character.
    output_text='<SOL> '+output_text+' <EOL>'
    input_resumes.append(input_text)
    output_summary.append(output_text)
    for word in input_text.split():
        if word not in input_tokens:
            input_tokens[word]+=1
    for word in output_text.split():
        if word not in output_tokens:
            output_tokens.add(word)
            
    file_count+=1
    if file_count==50:
        break
print("demmo file counts",dummy_file_count)

2
8
19
27
30
demmo file counts 5


In [3]:
print(len(input_tokens))
num_encoder_tokens=min(len(input_tokens),4000)
input_tokens=[word for word,count in input_tokens.most_common(num_encoder_tokens-1)]
input_tokens=sorted(list(input_tokens))
output_tokens=sorted(list(output_tokens))
num_decoder_tokens=len(output_tokens)
max_encoder_seq_len=max([len(text.split()) for text in input_resumes])
max_decoder_seq_len=max([len(text.split()) for text in output_summary])

print('Number of samples:',len(input_resumes))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)
print(output_summary[2])

2084
Number of samples: 50
number of unique input token: 2084
number of unique output token: 1229
Max Sequence length for inputs: 157
Max Sequence length for outputs: 147
<SOL>   <EOL>


## defining token2index

In [4]:
input_token2index=dict([(word,i) for i,word in enumerate(input_tokens)])
output_token2index=dict([(word,i) for i,word in enumerate(output_tokens)])
input_token2index['UNK']=num_encoder_tokens-1

## defing encoder_input,decoder_input and decoder_output

In [5]:
encoder_input_data=np.zeros((len(input_resumes),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_resumes),max_decoder_seq_len,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_resumes),max_decoder_seq_len,num_decoder_tokens),dtype='float32')


## creating training dataset

In [6]:
for i,(input_text,target_text) in enumerate(zip(input_resumes,output_summary)):
    for t,word in enumerate(input_text.split()[:max_encoder_seq_len]):
        if word not in input_token2index:
            word="UNK"
        encoder_input_data[i,t,input_token2index[word]]=1
        
    for t,word in enumerate(target_text.split()):
        decoder_input_data[i,t,output_token2index[word]]=1
        # decoder_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i,t-1,output_token2index[word]]=1


## Define an input sequence and process it.

In [7]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


## Set up the decoder, using encoder_states as initial state.

In [8]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Define the model that will turn
## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

In [9]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 2084)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 1229)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 1133056     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  695296      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

## function to test model

In [10]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_word_index = dict(
    (i, word) for word, i in input_token2index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in output_token2index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, output_token2index['<SOL>']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' '+sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '<EOL>' or
           len(decoded_sentence.split()) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    if('<EOL>' in decoded_sentence):
        decoded_sentence=" ".join(decoded_sentence.split()[:-1])
    return decoded_sentence


## Run training

In [ ]:
iteration_file="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_level/iteration_resume_level.txt"
iteration=0
try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    #print(iteration)
    file.close()
    # load weights
    print('loading the weights')
    file_path='/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_level/model/resume_level_iteration_'+str(iteration)+'.h5'
    model=load_model(file_path)
    # estimate accuracy on whole dataset using loaded weights
    scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
    print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
    print("Testing Samples\n"+"-"*50)
    print("-"*50)
    for i in range(1):
        index=np.random.randint(len(input_resumes))
        encoded_input_sequence=encoder_input_data[index: index + 1]
        output_sequence=decode_sequence(encoded_input_sequence)
        print("-"*50)
        print(input_resumes[index])
        print(" "*50)
        print("*"*50+"\nOUTPUT"+" "*50)
        print(output_sequence)

    
except:
    print('no file exist')


# checkpoint
filepath="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_level/checkpoints/resume_level_checkpoints.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    try:
        print('Iteration:',iteration+1)
        #training
        model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,callbacks=callbacks_list)
        #prepare sample_data to test 5 samples:
        print("-"*50)
        for i in range(1):
            index=np.random.randint(len(input_resumes))
            encoded_input_sequence=encoder_input_data[index: index + 1]
            output_sequence=decode_sequence(encoded_input_sequence)
            print("-"*50)
            print(input_resumes[index])
            print(" "*50)
            print("*"*50+"\nOUTPUT"+" "*50)
            print(output_sequence)

        # Save model
        file=open(iteration_file,'a')
        file.write('iteration:'+str(iteration+1)+'\n')
        file.close()
        file_path='/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_level/model/resume_level_iteration_'+str(iteration+1)+'.h5'
        model.save(file_path)
        iteration+=1
    
    except:
        file=open(iteration_file,'r')
        last_line=file.read().split('\n')[-2]
        print('file_data,',last_line)
        iteration=int(last_line.split(':')[1])
        #print(iteration)
        file.close()
        # load weights
        print('loading the weights')
        file_path='/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_level/model/resume_level_iteration_'+str(iteration)+'.h5'
        model=load_model(file_path)
        # estimate accuracy on whole dataset using loaded weights
        scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
        print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
        print("Testing Samples\n"+"-"*50)
        print("-"*50)
        for i in range(1):
            index=np.random.randint(len(input_resumes))
            encoded_input_sequence=encoder_input_data[index: index + 1]
            output_sequence=decode_sequence(encoded_input_sequence)
            print("-"*50)
            print(input_resumes[index])
            print(" "*50)
            print("*"*50+"\nOUTPUT"+" "*50)
            print(output_sequence)
        

file_data, iteration:39
loading the weights
acc: 40.31%


Testing Samples
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
  
abhinav sangal 
e-6 f, munirka, new delhi - 110067 
+91 9811798855(m), email: abhinavsangal@gmail.com 
objective 
highly accomplished, talented and professional marketing specialist with extensive knowledge of communication media and 
advertising. seeking for a position in client servicing and account management to enhance my professional experience and work for 
the growth of the organization. 
professional experience 
vdopia me dia & software pvt . ltd . octobe r 2013 – pre se nt 
senior client servicing & ad operations executive, gurgaon 
key responsibilities 
 client servicing and ad operations by ensuring perfect execution of advertising campaign for all clients. 
 good understanding of display, mobile and video ad campaign metrics and analysis, includi

/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_10 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_9_3/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'lstm_9_3/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/50
40/40 [==============================] - 7s 181ms/step - loss: 1.3815e-05 - acc: 0.4918 - val_loss: 6.3804 - val_acc: 0.0469
Epoch 3/50
40/40 [==============================] - 7s 173ms/step - loss: 1.2431e-05 - acc: 0.4918 - val_loss: 6.3816 - val_acc: 0.0469
Epoch 4/50
40/40 [==============================] - 7s 182ms/step - loss: 1.1208e-05 - acc: 0.4920 - val_loss: 6.3831 - val_acc: 0.0463
Epoch 5/50
40/40 [==============================] - 7s 185ms/step - loss: 1.0136e-05 - acc: 0.4918 - val_loss: 6.3848 - val_acc: 0.0463
Epoch 6/50
40/40 [==============================] - 8s 190ms/step - loss: 9.2039e-06 - acc: 0.4920 - val_loss: 6.3867 - val_acc: 0.0476
Epoch 7/50
40/40 [==============================] - 7s 175ms/step - loss: 8.3985e-06 - acc: 0.4918 - val_loss: 6.3885 - val_acc: 0.0483
Epoch 8/50
40/40 [==============================] - 7s 174ms/step - loss: 7.6895e-06 - acc: 0.4917 - val_loss: 6.3904 - val_acc: 0.0490
Epoch 9/50
40/40 [==============================

40/40 [==============================] - 7s 178ms/step - loss: 2.1538e-06 - acc: 0.4940 - val_loss: 6.4388 - val_acc: 0.0544
Epoch 2/50
40/40 [==============================] - 7s 173ms/step - loss: 2.2466e-06 - acc: 0.4922 - val_loss: 6.4172 - val_acc: 0.0469
Epoch 3/50
40/40 [==============================] - 7s 171ms/step - loss: 2.4861e-06 - acc: 0.4925 - val_loss: 6.3985 - val_acc: 0.0524
Epoch 4/50
40/40 [==============================] - 7s 176ms/step - loss: 2.4255e-06 - acc: 0.4929 - val_loss: 6.4410 - val_acc: 0.0537
Epoch 5/50
40/40 [==============================] - 7s 174ms/step - loss: 2.1160e-06 - acc: 0.4929 - val_loss: 6.4155 - val_acc: 0.0510
Epoch 6/50
40/40 [==============================] - 7s 179ms/step - loss: 2.0764e-06 - acc: 0.4930 - val_loss: 6.4286 - val_acc: 0.0537
Epoch 7/50
40/40 [==============================] - 7s 172ms/step - loss: 2.0949e-06 - acc: 0.4922 - val_loss: 6.4125 - val_acc: 0.0578
Epoch 8/50
40/40 [==============================] - 7s 176m

40/40 [==============================] - 8s 195ms/step - loss: 2.5053e-06 - acc: 0.4930 - val_loss: 6.4269 - val_acc: 0.0483
Epoch 2/50
40/40 [==============================] - 8s 211ms/step - loss: 2.4268e-06 - acc: 0.4932 - val_loss: 6.4366 - val_acc: 0.0483
Epoch 3/50
40/40 [==============================] - 8s 209ms/step - loss: 2.3561e-06 - acc: 0.4937 - val_loss: 6.4359 - val_acc: 0.0483
Epoch 4/50
40/40 [==============================] - 9s 221ms/step - loss: 2.2943e-06 - acc: 0.4940 - val_loss: 6.4366 - val_acc: 0.0490
Epoch 5/50
40/40 [==============================] - 8s 204ms/step - loss: 2.2406e-06 - acc: 0.4944 - val_loss: 6.4412 - val_acc: 0.0497
Epoch 6/50
40/40 [==============================] - 7s 187ms/step - loss: 2.1899e-06 - acc: 0.4954 - val_loss: 6.4460 - val_acc: 0.0517
Epoch 7/50
40/40 [==============================] - 7s 184ms/step - loss: 2.1513e-06 - acc: 0.4937 - val_loss: 6.4578 - val_acc: 0.0497
Epoch 8/50
40/40 [==============================] - 8s 194m

...

Iteration: 44
Train on 40 samples, validate on 10 samples
Epoch 1/50
30/40 [=====================>........] - ETA: 1s - loss: 1.8980e-06 - acc: 0.5660file_data, iteration:43
loading the weights


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-12-0499a980bf1c>", line 46, in <module>
    validation_split=0.2,callbacks=callbacks_list)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/training.py", line 1042, in fit
    validation_steps=validation_steps)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py", line 2661, in __call__
    return self._call(inputs)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py", line 2631, in _call
    fetched = self._callable_fn(*array_vals)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 1451, in __call__
    self._session._session, self._handle, args, status, None)
KeyboardInterru

TypeError: Can't convert 'list' object to str implicitly

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 636, in _abort